In [1]:
import sys
import time
import csv
import math
import numpy as np
import pyspark
from pyspark import SparkContext, SparkConf
import json
from operator import add
import pandas as pd
import xgboost as xgb
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from node2vec import Node2Vec as n2v
import networkx as nx
from hashlib import md5
import pickle
import os
import pandas as pd
from xgboost import XGBRegressor

pd.set_option("display.max_columns", 100)
pd.set_option('display.width', 100)
pd.set_option("display.precision", 2)


from utils import create_category_md5_mapping, integrate_mapping_user_bus_cat_data, dataframe_to_rdd_dict, analyze_top_business_categories, analyze_top_categories

import json
import numpy as np
from pyspark import SparkContext, SparkConf
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating

def initialize_spark_context(APP_NAME="Train: XGBModel"):
    # Spark配置项列表
    SPARK_CONF = [
        ("spark.dynamicAllocation.enabled", "true"),  # 启用动态资源分配
        ("spark.dynamicAllocation.maxExecutors", "10"),  # 最大执行器数量
        ("spark.executor.memory", "3g"),  # 每个执行器的内存
        ("spark.executor.cores", "2"),  # 每个执行器的CPU核心数
        ("spark.executor.memoryOverhead", "3000"),  # 执行器内存开销
        ("spark.driver.memory", "4g"),  # 驱动程序的内存
        ("spark.driver.maxResultSize", "2g"),  # 驱动程序的最大结果大小
        ("spark.python.worker.memory", "2g"),  # Python工作进程的内存
        ("spark.sql.shuffle.partitions", "20"),  # Shuffle操作的分区数
        ("spark.sql.sources.partitionOverWriteMode", "dynamic"),  # 分区覆写模式
        ("spark.network.timeout", "600s"),  # 网络超时设置
        ("spark.executor.heartbeatInterval", "120s"),  # 执行器心跳间隔
    ]

    # 创建Spark配置
    spark_conf = pyspark.SparkConf()
    spark_conf.setAppName(APP_NAME)
    spark_conf.setAll(SPARK_CONF)

    # 创建SparkContext
    sc = pyspark.SparkContext(conf=spark_conf)
    sc.setLogLevel("ERROR")  # 设置日志级别

    return sc

sc = initialize_spark_context()

24/04/20 15:28:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [6]:
!pip install nltk

     |████████████████████████████████| 1.5 MB 4.1 MB/s eta 0:00:01
     |████████████████████████████████| 776 kB 16.8 MB/s eta 0:00:01


In [2]:
import better_features
from better_features import FeatureProcessor, read_json_data, transform_user_data, transform_business_data, extract_review_data
from datetime import datetime #add
import time
import pandas as pd

def rdd_to_pandas(rdd):
    return pd.DataFrame(rdd.collect(), columns=rdd.first().keys())

start_time = time.time()

user_parsed_df = pd.read_csv('cache/user_df.csv') # parsed from users.json
business_parsed_df = pd.read_csv('cache/business_df.csv') # parsed from business.json
review_parsed_df = pd.read_csv('cache/review_df.csv') # parsed from business.json

print("DONE!")

DONE!


In [3]:
from better_features import FeatureProcessor, read_json_data, transform_user_data, transform_business_data,extract_review_data
from utils import integrate_mapping_user_bus_cat_data
from KMeans_user_cluster import KMeans_process_user_clusters


feature_processor = FeatureProcessor(user_parsed_df, business_parsed_df, review_parsed_df)

# Kmeans find clusters for user and biz, here are some mappings to be used
user_clusters, clusters_important_cities, business_with_important_cat_df = KMeans_process_user_clusters(feature_processor.map_reviews_with_categories(), business_parsed_df)

/usr/local/lib/python3.8/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [6]:
# ## TRAIN DATA FEATURE PROCESSING HERE
# train_data_file = '../yelp_combined.csv'
# train_df = pd.read_csv(train_data_file)

# data_folder_path = '../data/'
# res_rdd = feature_processor.process_all_features(sc, train_df, data_folder_path, train_data_file)

# final_df = rdd_to_pandas(res_rdd)
# final_df = final_df.merge(user_clusters, on='user_id', how='left')

# ## TEST DATA FEATURE PROCESSING HERE
# test_data_file = '../yelp_true.csv'
# test_df = pd.read_csv(test_data_file)

# test_res_rdd = feature_processor.process_all_features(sc, test_df, data_folder_path, test_data_file)

# val_df = rdd_to_pandas(test_res_rdd)
# val_df = val_df.merge(user_clusters, on='user_id', how='left')

## START

In [8]:
review_df= pd.read_csv('cache/review_df.csv')

In [9]:
import spacy
from spacy.cli import download

import nltk
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')

# download('en_core_web_sm')
# nlp = spacy.load('en_core_web_sm', disable=["parser", "ner"])

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [14]:
import re
# import inflect
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

def normalize_lemmatize(words):
    lemmatizer = nltk.stem.WordNetLemmatizer()
    
    return [lemmatizer.lemmatize(word.lower()) for word in words if word.isalpha()]

In [15]:
def get_processed_with_spark(sample, sc):
    # 直接从Pandas DataFrame创建RDD
    rdd = sc.parallelize(sample[['business_id', 'text']].values.tolist())

    # 定义转换函数
    def process_row(row):
        business_id, text = row  # 直接解包元组
        words = nltk.word_tokenize(text)
        processed_text = ' '.join(normalize_lemmatize(words))
        return (business_id, processed_text)

    processed_rdd = rdd.map(process_row)
    return processed_rdd

def get_review_processed_with_spark(processed_rdd, reviews_rdd, sc):
    # 使用broadcast优化查找过程
    processed_business_ids = sc.broadcast(set(processed_rdd.map(lambda x: x[0]).collect()))
    
    # 使用filter过滤reviews_rdd
    reviews_processed_rdd = reviews_rdd.filter(lambda x: x['business_id'] in processed_business_ids.value)
    return reviews_processed_rdd

In [16]:
processed_rdd = get_processed_with_spark(review_df, sc)

In [21]:
# Make sure not overwrite, Already saved.
try:
    processed_rdd.saveAsTextFile('cache/processed_review_text')
    print("Data saved successfully!")
except Exception as e:
    print("Error saving data: ", e)

In [33]:
processed_rdd.take(1)

[('iKMLsX1Je7P3wAOEc9scDg',
  'i have been itching to get to origin for month now after a friend of mine had gone and raved about the food i dug the concept tapa with a twist this is definitely not your traditional spanish tapa at origin you sample small plate with asian italian and latin influence i did find the menu overwhelming but our server wa more than happy to help steer the meal so that similar and complimentary flavour were eaten together it wa tough to narrow it down a we would have been happy with just about anything on the menu we started with the tostones smashed flattened and plantain with guacamole on the side the plantain were very well seasoned with generous use of kosher salt and a light dusting of curry powder next up were the deviled egg with smoked bacon and gremolata and the smoked cod croquette with saffron aioli both were a big hit at the table disappearing in a flash i preferred the croquette of the two but i did exactly not eat my share of the deviled egg eith

Exception in thread "serve RDD 14" java.net.SocketTimeoutException: Accept timed out
	at java.net.PlainSocketImpl.socketAccept(Native Method)
	at java.net.AbstractPlainSocketImpl.accept(AbstractPlainSocketImpl.java:409)
	at java.net.ServerSocket.implAccept(ServerSocket.java:560)
	at java.net.ServerSocket.accept(ServerSocket.java:528)
	at org.apache.spark.security.SocketAuthServer$$anon$1.run(SocketAuthServer.scala:64)


## LOAD RDD ADN START HERE

In [ ]:
# Load RDD here

processed_rdd = sc.textFile('cache/processed_review_text')

In [35]:
# import gensim.downloader as api
# from gensim.models import Word2Vec
# from gensim.models import KeyedVectors

# # 加载预训练的Word2Vec模型
# word_vectors = api.load("word2vec-google-news-300")

In [36]:
# def sentence_vector(sentence):
#     words = sentence.split()
#     if len(words) >= 1:
#         return np.mean([model[word] for word in words if word in model], axis=0)
#     else:
#         return np.zeros(100)  # 假设词向量长度为100

In [34]:
# review_df['sentence_vec'] = review_df['processed_text'].apply(sentence_vector)